Server Information:
- the version of the notebook server is 4.3.1 and is running on: Python 3.6.0 |Anaconda 4.3.0 (64-bit)
- current Kernel Information: Python 3.6.0 |Anaconda 4.3.0 (64-bit)| IPython 5.1.0
- MongoDB shell version: 2.6.10

###### report

# DATA WRANGLING WITH MONGODB

Map Area: Khmelnytskyi raion (a district) of the Khmelnytskyi Oblast in western Ukraine.

- https://www.openstreetmap.org/relation/1739099
- http://overpass-api.de/api/map?bbox=26.5306,49.2906,27.3257,49.6103 
- https://en.wikipedia.org/wiki/Khmelnytskyi_Raion

I live in this place, and I want to play with requests to this database.
I want to contribute to improving the map of my hometown at OpenStreetMap.org.

## Preliminary analysis

For the preliminary analysis of the data, I will create a dictionary that includes all records of osm-file where the type of elements is the tag. The key of a tag will be the key of the dictionary, and the value will be a set that will include all the values of tags.

###### background

In [162]:
import xml.etree.cElementTree as ET
from pymongo import MongoClient
from collections import defaultdict
import json
import codecs
import pprint

file_input = 'khmelnytskyi_region.osm'
file_output = 'khmelnytskyi_region.json'

In [163]:
def accumulate_tags(filename):
    # accumulates content of all tags in the dictionary.
    content = defaultdict(set)
    for _, element in ET.iterparse(filename):
        if element.tag == 'tag':
            content[element.attrib['k']].add(element.attrib['v'])
    return content

def check_out(keys, verified_keys):
    # shows the result of check
    print(list(verified_keys))
    keys -= verified_keys
    print('\nverified keys:', len(verified_keys))
    print('the remainder of keys:', len(keys))

In [164]:
content = accumulate_tags(file_input)

In [165]:
# set of tags names
keys = set(content.keys())
len(content)

442

###### report

The dictionary has 442 entries that I'm going to explore.

###### background

In [166]:
# int check
verified_keys = set()
for key in keys:
    verified_keys.add(key)
    for item in content[key]:
        try:
            int(item)
        except:
            verified_keys.remove(key)
            break

check_out(keys, verified_keys)

['passenger_lines', 'koatuu', 'start_date', 'tracks', 'capital', 'population', 'level', 'frequency', 'number', 'ISO3166-1:numeric', 'mtb:scale', 'cables', 'ele', 'maxspeed', 'rooms', 'esr:code', 'admin_level', 'layer', 'capacity', 'express:code', 'circuits', 'lanes', 'platforms', 'building:levels', 'whc:inscription_date', 'gauge', 'esr:user', 'stars', 'heritage', 'height']

verified keys: 30
the remainder of keys: 412


###### report

- 30 tags converted into integers

###### background

In [167]:
# float check
verified_keys = set()
for key in keys:
    verified_keys.add(key)
    for item in content[key]:
        try:
            float(item)
        except:
            verified_keys.remove(key)
            break

check_out(keys, verified_keys)

['distance', 'width']

verified keys: 2
the remainder of keys: 410


###### report

- Two tags converted into float numbers

###### background

In [168]:
# languages check
verified_keys = set()
for key in keys:
    if key.startswith('name:') and len(key) == 7 and key not in ('name:uk', 'name:en'):
        verified_keys.add(key)
        
check_out(keys, verified_keys)

['name:su', 'name:mi', 'name:ce', 'name:am', 'name:hr', 'name:yi', 'name:ug', 'name:eu', 'name:km', 'name:dv', 'name:ar', 'name:tg', 'name:lt', 'name:tk', 'name:tl', 'name:nn', 'name:et', 'name:mk', 'name:ne', 'name:he', 'name:is', 'name:fa', 'name:ee', 'name:ca', 'name:bg', 'name:cv', 'name:ss', 'name:af', 'name:ig', 'name:kk', 'name:ro', 'name:or', 'name:jv', 'name:gl', 'name:sm', 'name:mr', 'name:rm', 'name:kv', 'name:sh', 'name:io', 'name:la', 'name:pt', 'name:pl', 'name:kg', 'name:sl', 'name:cs', 'name:qu', 'name:oc', 'name:gv', 'name:ps', 'name:zh', 'name:bi', 'name:ka', 'name:na', 'name:ru', 'name:kw', 'name:cy', 'name:mt', 'name:yo', 'name:ko', 'name:ky', 'name:ku', 'name:wo', 'name:ab', 'name:ia', 'name:da', 'name:cu', 'name:tr', 'name:kn', 'name:ba', 'name:be', 'name:kl', 'name:fr', 'name:uz', 'name:sn', 'name:fy', 'name:es', 'name:lv', 'name:my', 'name:ml', 'name:sk', 'name:rw', 'name:it', 'name:br', 'name:lb', 'name:li', 'name:id', 'name:bo', 'name:gn', 'name:ms', 'name:bn'

###### report

- 133 tags rejected because they are names of objects in languages that I don't know.

###### background

In [169]:
# len == 1 check
verified_keys = set()
for key in keys:
    if len(content[key]) == 1:
        verified_keys.add(key)

check_out(keys, verified_keys)

['name:mdf', 'maxstay', 'branch:ru', 'name:pcd', 'timezone', 'nat_ref', 'name:sah', 'name:bar', 'embankment', 'atm', 'name:nds-nl', 'psv', 'name:cdo', 'contact:email', 'location', 'toilets:disposal', 'addr:country', 'ISO3166-2', 'recycling:paper', 'clothes', 'name:nov', 'name:lez', 'generator:output:hot_water', 'name:haw', 'name:ceb', 'name:zh-classical', 'name:chy', 'name:simple', 'education', 'name:tum', 'building:parts', 'alt_name:de', 'name:udm', 'waste', 'sac_scale', 'unisex', 'heritage:operator', 'name:nrm', 'name:ext', 'name:lmo', 'name:dsb', 'lawyer', 'boundary', 'fuel:octane_91', 'noname', 'name:hsb', 'name:ilo', 'name:fiu-vro', 'name:pap', 'section', 'name:gag', 'name:vep', 'name:jbo', 'name:gan', 'contact:phone', 'bus', 'name:mzn', 'destination', 'traffic_calming', 'name:mhr', 'name:tpi', 'ISO3166-1', 'name:chr', 'name:frp', 'name:ast', 'delivery', 'currency:USD', 'alt_name:eo', 'name:lad', 'traffic_sign', 'ISO3166-1:alpha3', 'name:krc', 'old_name:vi', 'traffic_signals', 'na

###### report

- 163 tags rejected because they have only one value.

###### background

Now, I compare remainder of tags with the description of the OSM-wiki. I uploaded the file **taginfo-wiki.db.bz2**(data from the tag and key pages of the OSM wiki) from the site [taginfo](https://taginfo.openstreetmap.org/download) and transformed him to a dictionary.

In [170]:
# select
import sqlite3
conn = sqlite3.connect('taginfo-wiki.db')
c = conn.cursor()
taginfo_wiki = c.execute('''
    SELECT DISTINCT key, value, description 
    FROM wikipages 
    WHERE lang="en" 
    ORDER BY key, value;
''')

In [171]:
# accumulates wiki tags in the dictionary
wiki = dict()
old_key = None
for key, value, description in taginfo_wiki:
    if old_key != key:
        old_key = key
        wiki[key] = dict()
        wiki[key]['values'] = set()
        wiki[key]['description'] = description
    wiki[key]['values'].add(value)

In [172]:
# tag versus OSM-wiki
verified_keys = set()
report = []
for key in keys:
    desc = None
    if key in wiki.keys():
        desc = wiki[key]['description']
        if not content[key] - wiki[key]['values']:
            verified_keys.add(key)
            continue
    report.append((len(content[key]), key, desc))

check_out(keys, verified_keys)    

['sport', 'power', 'public_transport', 'entrance', 'aeroway', 'wood', 'bicycle', 'historic', 'tower:type', 'motorcycle', 'junction', 'oneway', 'waterway', 'natural', 'highway', 'railway', 'place', 'surface', 'route', 'shelter', 'tourism', 'tunnel', 'footway', 'man_made', 'religion', 'dispensing', 'building', 'foot', 'leisure', 'information', 'substation', 'wheelchair', 'lit']

verified keys: 33
the remainder of keys: 81


###### report

- 33 tags rejected because they have values that equal to values from osm-wiki and contains nothing superfluous.

**81 tag left to handle manually**

###### background

In [173]:
pprint.pprint(sorted(report, reverse=True))

[(1651, 'name', 'The primary tag used for naming an element.'),
 (685, 'addr:housenumber', None),
 (623, 'name:uk', None),
 (576, 'name:en', None),
 (179, 'phone', 'A telephone number associated with the object.'),
 (158,
  'wikipedia',
  "Provides a link to Wikipedia's article about the feature."),
 (150, 'wikidata', 'The ID of the Wikidata item about the feature.'),
 (134, 'addr:street', None),
 (133,
  'opening_hours',
  'Describes when something is open or closed.  There is a specific standard '
  'format for this data.'),
 (130, 'website', 'Specifying the link to the official website for a feature.'),
 (127, 'wikipedia:ru', None),
 (90,
  'ref',
  'Used for reference numbers or codes. Common for roads, highway exits, '
  'routes, etc.'),
 (89, 'addr:postcode', None),
 (87,
  'description',
  'To provide additional information about the related element to the end map '
  'user.'),
 (62, 'shop', 'A place selling retail products or services.'),
 (58, 'old_name', None),
 (52,
  'ameni

###### report

## Solving problems in the tags

### tag 'addr:housenumber'

In [174]:
list(sorted(content['addr:housenumber']))[319:325]

['29/1', '2А', '2а', '3', '3/1', '3/2']

House numbers include digits and letters. They have large and small letters and also have Latin letters instead of Cyrillic.

In [175]:
def addr_housenumber(item):
    """ The function converts house number to uppercase
        and replaces Latin letter 'A' to Cyrillic letter 'А'.

    Args:
        item (str): house number

    Returns:
        str: fixed house number    
    """
    return item.upper().replace('A', 'А') 

###### background

In [176]:
# after treatment
print('dirty house numbers', len(content['addr:housenumber']))
print('clean house numbers', len(set([addr_housenumber(item) for item in content['addr:housenumber']])))

dirty house numbers 685
clean house numbers 680


###### report

### tags 'name', 'description' and 'addr:street'

In [177]:
[item for item in content['addr:street'] if '.' in item]

['вул.Лісогринівецька', 'пр.Миру 102/2\n', 'пр. Миру']

Street names can contain abbreviations and special marks.

In [178]:
def addr_street(item):
    """ The function deletes separator (\n, \t) from a name of a street
        and replaces Cyrillic abbreviations to the full name.

    Args:
        item (str): name of a street
        
    Returns:
        str: fixed name of a street    
    """
    item = ' вулиця '.join(item.split('вул.'))
    item = ' проспект '.join(item.split('пр.'))
    return ' '.join(item.split())

###### background

In [179]:
# after treatment
[addr_street(item) for item in content['addr:street'] if '.' in item]

['вулиця Лісогринівецька', 'проспект Миру 102/2', 'проспект Миру']

###### report

### tag 'addr:city'

In [180]:
content['addr:city']

{'Гвардійське',
 'Хмельницкий',
 'Хмельницький',
 'Чорний Острів',
 'ст. Богданівці',
 'хмельницкий',
 'хмельницький'}

The name of the two cities recorded twice. The first time the name starts with a capital letter a second time with a small letter. And one name contains abbreviation.

In [181]:
def addr_city(item):
    """ The function capitalizes Cyrillic letter 'х' in the start of the word
        and replaces Cyrillic abbreviations to the full name.

    Args:
        item (str): name of a city
        
    Returns:
        str: fixed name of a city
    """    
    if item.startswith('ст.'):
        item = 'станція' + item[3:]
    elif item.startswith('х'):
        item = 'Х' + item[1:]
    return item

###### background

In [182]:
# after treatment
set([addr_city(item) for item in content['addr:city']])

{'Гвардійське',
 'Хмельницкий',
 'Хмельницький',
 'Чорний Острів',
 'станція Богданівці'}

###### report

### tag 'phone' and 'phone_1'

In [183]:
lens = set(range(100))
for phone in content['phone']:
    if len(phone) in lens:
        lens.remove(len(phone))
        print(phone)

+380382651217
(0382) 777-832
+38 0382 74-20-87
+38 0382 784767
0382 702030, 067 3831328
+30382701010
+38 (03822) 3-2241
+38 (050) 436-19-45
0382704903
0 (97) 110 50 69
719700
64-74-84


In [184]:
# What symbols did use to enter phone numbers?
print(list(sorted(set(i for item in content['phone'] for i in item))))

[' ', '(', ')', '+', ',', '-', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [185]:
# phone nubera with '/' and ','
[item for item in content['phone'] if ('/' in item) or (',' in item)]

['0382 702030, 067 3831328',
 '65-74-74 / 65-10-91',
 '067-311-81-95 / 71-86-45',
 '067-327-40-67 / 65-54-93']

Some items have delimiters and contain two phone numbers. Phone numbers should be in international ([ITU-T E.164](http://wiki.openstreetmap.org/wiki/Key:phone)) format.

In [186]:
def format_phone(phone):
    """ The function converts phone number to international (ITU-T E.164) format
        following the pattern '+<country code> <area code> <local number>'
        http://wiki.openstreetmap.org/wiki/Elements

    Args:
        item (str): numbers of phones with delimiters ',' or '/'

    Returns:
        list of str: corrected phone number
    """
    phones = phone.replace('/', ',').split(',')
    for i in range(len(phones)):
        item = ''.join(list(reversed([x for x in phones[i] if x.isdigit()])))
        if len(item) > 7:
            item = item[:7] + ' ' + item[7:]
        if len(item) > 11:
            item = item[:11] + ' ' + item[11:]
        if len(item) > 12:
            item = item + '+'
        phones[i] = item[::-1]
    return phones

###### background

In [187]:
# after treatment
lens = set(range(100))
for phone in content['phone']:
    phone = format_phone(phone)
    if len(phone[0]) in lens:
        lens.remove(len(phone[0]))
        print(phone)

['+38 038 2651217']
['038 2777832']
['+3 038 2701010']
['657474', '651091']


###### report

### tag 'addr:postcode'

In [188]:
[item for item in content['addr:postcode'] if len(item) != 5]

['29000-29499', '2900']

One postcode has the error less than five characters. Another postcode is the range of codes entered in one field. I will not import these postcodes.

Now I ready to make json-file and export this file in MongoDB

###### background

In [189]:
## clearing unused variables
# content.clear()
# report.clear()

In [190]:
def shape_element(element):
    """ This procedure generates one document for related elements.

    Args:
        element: one node from OSM-file with subordinate elements

    Returns:
        dict: JSON-document for MongoDB
    """
    
    phone_name_suffix = [''] + ['_'+str(x) for x in range(1, 10)]
    phone_name_number = 0
    
    document = {}
    if element.tag in ['node', 'way', 'relation']:
        document["_id"] = {'$numberLong': element.attrib['id']}
        document["type"] = element.tag
        document['user'] = element.attrib['user']
        document['uid'] = int(element.attrib['uid'])
        document['version'] = int(element.attrib['version'])
        document['changeset'] = int(element.attrib['changeset'])
        document['timestamp'] = {"$date": element.attrib['timestamp']}
        if element.tag == 'node':
            document['location'] = {"type": "Point",
                                    "coordinates": [float(element.attrib['lon']),
                                                    float(element.attrib['lat'])] }        

        document['tags'] = dict()
        document['refs'] = list()
        document['members'] = list()
                
        for child in element:
            if child.tag == 'tag':
                k = child.attrib['k']
                v = child.attrib['v']
                if k in ['ele', 'esr:code', 'heritage', 'gauge', 'cables', 'koatuu',
                         'ISO3166-1:numeric', 'whc:inscription_date', 'capital',
                         'lanes', 'height', 'start_date', 'platforms', 'layer',
                         'level', 'frequency', 'admin_level', 'building:levels',
                         'capacity', 'passenger_lines', 'stars', 'express:code',
                         'tracks', 'esr:user', 'mtb:scale', 'number', 'circuits',
                         'maxspeed', 'population', 'rooms']:
                    document['tags'][k] = int(v)

                elif k in ['width', 'distance']:
                    document['tags'][k] = float(v)

                elif k == 'addr:housenumber':
                    document['tags'][k] = addr_housenumber(v)

                elif k in ['name', 'description', 'addr:street']:
                    document['tags'][k] = addr_street(v)

                elif k == 'addr:city':
                    document['tags'][k] = addr_city(v)

                elif k == 'addr:postcode':
                    if len(v) == 5:
                        document['tags'][k] = int(v)

                elif k in ['phone', 'phone_1']:
                    for phone in format_phone(v):
                        phone_key = 'phone' + phone_name_suffix[phone_name_number]
                        document['tags'][phone_key] = phone
                        phone_name_number += 1

                else:
                    document['tags'][k] = v

            if child.tag == 'nd':
                document['refs'].append({'$numberLong': child.attrib['ref']})
                
            if child.tag == 'member':
                member = dict()
                for k, v in child.items():
                    if k == 'ref':
                        member[k] = {'$numberLong': v}
                    else:
                        member[k] = v
                document['members'].append(member)
                
        # clearing unused variables                
        if not document['tags']:
            del document['tags']
        if not document['refs']:
            del document['refs']
        if not document['members']:
            del document['members']

        return document
    else:
        return None

In [214]:
def process_map(file_xml, file_json):
    """ The function converts OSM-file to JSON-file

    Args:
        file_xml (str): name of OSM-file for reading
        file_json (str): name of JSON-file for writing

    Returns:
        None
    """
    with codecs.open(file_json, "w") as fo:
        for _, element in ET.iterparse(file_xml, events=("start",)):
            el = shape_element(element)
            if el:
                fo.write(json.dumps(el) + "\n")

#### uncomment next block for recording JSON-file

In [216]:
# process_map(file_input, file_output)

**This command imports JSON-file to MongoDB.**

```bash
$ mongoimport -db maps -c khmelnytskyi --file khmelnytskyi_region.json --jsonArray
```

In [193]:
# connect with MongoDB
client = MongoClient('localhost:27017')
db = client['maps']

# results of executing aggregation pipeline operators
def aggregate(pipeline):
    return pprint.pprint(list(db.khmelnytskyi.aggregate(pipeline)))

###### report

## Data Overview

                                                
### File sizes

In [194]:
!stat --printf='%s bytes  -  %n\n' khmelnytskyi*

110144963 bytes  -  khmelnytskyi_region.json
74945880 bytes  -  khmelnytskyi_region.osm


### About the dataset

##### Number of documents

In [195]:
db.khmelnytskyi.count()

388520

##### Number of documents by type

In [196]:
aggregate([
    {'$group':{'_id':'$type', 'count':{'$sum': 1}}},
    {'$sort':{'_id': 1}}
])

[{'_id': 'node', 'count': 349488},
 {'_id': 'relation', 'count': 460},
 {'_id': 'way', 'count': 38572}]


##### Number of unique users

In [197]:
len(db.khmelnytskyi.distinct("user"))

263

##### Top 5 contributing user

In [198]:
aggregate([
    {'$group':{'_id':'$user', 'count':{'$sum': 1}}},
    {'$sort':{'count': -1}},
    {'$limit': 5}
])

[{'_id': 'georg troyan', 'count': 247803},
 {'_id': 'teologovsan', 'count': 37627},
 {'_id': 'Vulpes-Vulpeos', 'count': 22903},
 {'_id': 'dexter_khm', 'count': 18667},
 {'_id': 'InnerIn', 'count': 15889}]


##### Number of users appearing only once (having 1 post)

In [199]:
aggregate([
    {'$group':{'_id':'$user', 'count':{'$sum': 1}}},
    {'$match':{'count': {'$eq': 1}}},
    {'$group': {'_id': 'having 1 post', 'count':{'$sum': 1}}}
])

[{'_id': 'having 1 post', 'count': 66}]


##### User of the year

In [200]:
aggregate([
    {'$group':{
        '_id':{'year': { '$year': '$timestamp' }, 'user': '$user'},
        'count':{'$sum': 1}
    }},
    { '$sort': { '_id.year': 1, 'count': -1 }},
    { '$group':{
        '_id':'$_id.year',
        'first': {'$first': {'user': '$_id.user','count': '$count'}}
    }},
    { '$sort': { '_id': 1}}
])

[{'_id': 2008, 'first': {'count': 74, 'user': 'Sturla'}},
 {'_id': 2009, 'first': {'count': 617, 'user': 'arconaut'}},
 {'_id': 2010, 'first': {'count': 1306, 'user': 'arconaut'}},
 {'_id': 2011, 'first': {'count': 37627, 'user': 'teologovsan'}},
 {'_id': 2012, 'first': {'count': 8628, 'user': 'georg troyan'}},
 {'_id': 2013, 'first': {'count': 60846, 'user': 'georg troyan'}},
 {'_id': 2014, 'first': {'count': 170587, 'user': 'georg troyan'}},
 {'_id': 2015, 'first': {'count': 7742, 'user': 'georg troyan'}},
 {'_id': 2016, 'first': {'count': 15889, 'user': 'InnerIn'}},
 {'_id': 2017, 'first': {'count': 7015, 'user': 'Vulpes-Vulpeos'}}]


##### Number of data entered by a year

In [201]:
aggregate([
    {'$group':{
        '_id': { '$year': '$timestamp' }, 'count':{'$sum': 1}
    }},
    {'$sort': { '_id': 1}}
])

[{'_id': 2008, 'count': 74},
 {'_id': 2009, 'count': 656},
 {'_id': 2010, 'count': 1410},
 {'_id': 2011, 'count': 58483},
 {'_id': 2012, 'count': 16947},
 {'_id': 2013, 'count': 71637},
 {'_id': 2014, 'count': 175072},
 {'_id': 2015, 'count': 18177},
 {'_id': 2016, 'count': 38620},
 {'_id': 2017, 'count': 7444}]


##### Number of changeset with only one changed element

A changeset consists of a group of changes made by a single user over a short period of time.

In [202]:
aggregate([
    {'$group': {'_id': '$changeset', 'count': {'$sum': 1}}},
    {'$match': {'count': 1}},
    {'$group': {'_id': 'only one change', 'count': {'$sum': 1}}}
])

[{'_id': 'only one change', 'count': 989}]


##### Largest changeset in this region

In [203]:
aggregate([
    {'$group': {'_id': '$changeset', 'elemens': {'$sum': 1}}},
    {'$sort': {'elemens': -1}},
    {'$limit': 1}
])

[{'_id': 9366778, 'elemens': 4914}]


##### Number of new elements in the largest changeset

In [204]:
aggregate([
    {'$match': {'changeset': 9366778, 'version': 1}},
    {'$group': {'_id': 'new elements', 'count': {'$sum': 1}}}
])

[{'_id': 'new elements', 'count': 15}]


##### Number of changes by type in the largest changeset

In [205]:
aggregate([
    {'$match': {'changeset': 9366778}},
    {'$group': {'_id': '$type', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}}
])

[{'_id': 'node', 'count': 4910},
 {'_id': 'way', 'count': 3},
 {'_id': 'relation', 'count': 1}]


### About the tags

##### Top 3 appearing amenities

In [206]:
aggregate([
    {'$match': {'tags.amenity' : { '$exists': True } }},
    {'$group': {'_id': '$tags.amenity', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}},
    {'$limit': 3}
])

[{'_id': 'parking', 'count': 149},
 {'_id': 'hospital', 'count': 65},
 {'_id': 'school', 'count': 59}]


##### Top 3 shops

In [207]:
aggregate([
    {'$match': {'tags.shop' : { '$exists': True } }},
    {'$group': {'_id': '$tags.shop', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}},
    {'$limit': 3}
])

[{'_id': 'convenience', 'count': 79},
 {'_id': 'supermarket', 'count': 30},
 {'_id': 'kiosk', 'count': 23}]


##### Top 5 appearing name's of street

In [208]:
aggregate([
    {'$match': {'tags.addr:street' : { '$exists': True } }},
    {'$group': {'_id': '$tags.addr:street', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}},
    {'$limit': 5}
])

[{'_id': 'Проскурівська вулиця', 'count': 138},
 {'_id': 'Миру проспект', 'count': 84},
 {'_id': 'Подільська вулиця', 'count': 81},
 {'_id': 'Грушевського вулиця', 'count': 81},
 {'_id': 'Соборна вулиця', 'count': 76}]


##### Top 3 type's of leisure

In [209]:
aggregate([
    {'$match': {'tags.leisure' : { '$exists': True } }},
    {'$group': {'_id': '$tags.leisure', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}},
    {'$limit': 3}
])

[{'_id': 'pitch', 'count': 71},
 {'_id': 'garden', 'count': 62},
 {'_id': 'stadium', 'count': 16}]


### About the map

##### If I want to visit the central park of Khmelnitsky where can I charge my car at this time?

In [210]:
aggregate([
    {'$geoNear': {
        'near': { 'type': 'Point', 'coordinates': [ 26.97257, 49.43154 ] },
        'distanceField': 'distance',
        'query': {'tags.amenity': 'charging_station'},
        'includeLocs': "location",
        'limit': 3,
        'spherical': True
    }},
    {'$project':{'_id': 0, 'distance':1, 'location.coordinates': 1}}
])

[{'distance': 814.3856324451427,
  'location': {'coordinates': [26.9801727, 49.4261483]}},
 {'distance': 847.8606041482592,
  'location': {'coordinates': [26.9812475, 49.4264254]}},
 {'distance': 1262.6767588361008,
  'location': {'coordinates': [26.981175, 49.4216741]}}]


The best choice is number two, and I can find the [way from charging station to the park](https://www.openstreetmap.org/directions?engine=mapzen_foot&route=49.4264254%2C26.9812475%3B49.43154%2C26.97257).

## Conclusion

At first glance, the data are the result of the work of communities contain many errors and for checking their need much time. But really, everything was easier.

The attributes of elements had no problems. Only geolocation data required the particular form for record data to JSON-file.

Then I investigated 442 kinds of tags. At first, I rejected tags that can be automatically converted to numbers. Then I dismissed the tags that have only one value and tags of objects in foreign languages that I do not know. 

Check the tags using external sources also brought success. To test the data for loading into MongoDB, I used the SQLite database from the [taginfo.openstreetmap.org](taginfo.openstreetmap.org) site and rejected 33 tags that thoroughly described in the wiki and did not contain unnecessary information. 

As a result, 81 tags left to handle manually, of which only 33% contained more than ten values. 

After loading data into MongoDB and viewing statistics, it appears that information about the region in which I live is not complete. Very few objects are attractive to consumers. I think the reason for this is the poor activity of communities. Only 3 of the 263 users contributed almost 80% of the information. Peak data entry was in 2014 and now decreases.

But the primary goal achieved - information from an open source that made by communities can prepay and use for various tasks. Particularly, in this case,  I interested to make requests about location data and calculating distances on the map.

## What can be done to improve the data set?

If we consider the specific data set of the map, we can use the [Notes API](http://wiki.openstreetmap.org/wiki/Notes#Notes_API) to load comments to incomplete data (for example, for the houses that are marked on the map and not have a number ask the number) and request the community to assist in completing the required data.

Advantages:

- the corrected data immediately sends on the site, and this information will be available for the next download
- Notes API allow tracking how the community is correcting data and seeing which items are closed and are not.

Disadvantages:

- the community can not immediately edit all notes, it may take a long time, and all data will not ready to the desired time.
- the fixed data must again pass through the procedure of preparation, review, and download.

If I will dream about improving the data on the site, it is possible to develop a game similar to Pokemon GO app that will combine data from maps and stream of the camera. The goal of the game, instead of collecting Pokemon hunt the white spots on the map and bring information about them to the site using a GPS. After the introduction of related information about objects, gamers gain points and become the owners of the facility in the virtual space. They can sell them or exchange on other artifacts. Players can unite in groups. They may fight among themselves. They may hunt the new facilities on the map, may fix inaccuracies of other users or add new relevant related information of old objects.

There may be not active users who can watch the results of the game. For example pointing a smartphone at a building, to see the information about him. The picture is something like the Terminator seen: house number, owner, information about companies inside the building, and other.

Advantages:

- invitation new members of the community
- data will input directly from the local place using GPS

Disadvantages:
- collected data may contain many inaccuracies because the time affect the outcome of the game 
- Received data belong to certain classes of objects that will be limited the size of smartphone's screen. Information about buildings will be entered easy but will complex task collect data for roads, forests, lakes and other big objects.
- will be hard to maintain the activity of players when white spots on the maps will tend to end

## References

- [OpenStreetMap](https://www.openstreetmap.org/)
- [OpenStreetMap Documentation](http://wiki.openstreetmap.org/wiki/Main_Page)
- [OSM XML Documentation](http://wiki.openstreetmap.org/wiki/OSM_XML)
- [Taginfo download page](https://taginfo.openstreetmap.org/download)
- [ITU-T E.164 - international format for numbers phone](http://wiki.openstreetmap.org/wiki/Key:phone)
- [Notes API](http://wiki.openstreetmap.org/wiki/Notes#Notes_API)


###### background

## Tools for cleaning report

In [211]:
def clean_report(in_file, out_file):
    # remove background parts of the report
    out = False
    with open(in_file, 'r') as f_in:
        with open(out_file, 'w') as f_out:
            for line in f_in:
                if line.strip().startswith('###### report'):
                    out = True
                elif line.strip().startswith('###### background'):
                    out = False
                else:
                    if out:
                        f_out.write(line)

In [212]:
# convert to md-file
!jupyter nbconvert --to markdown investigation.ipynb

[NbConvertApp] Converting notebook investigation.ipynb to markdown
[NbConvertApp] Writing 37077 bytes to investigation.md


In [213]:
clean_report('investigation.md', 'report.md')